## 가변 길이 인수(역전파 편)


### 13.1 가변 길이 인수에 대응한 Add 클래스의 역전파

### colab 접속

In [1]:
if __name__ == '__main__':
  !pip install import_ipynb
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/drive/MyDrive/밑바닥부터 시작하는 딥러닝3

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp37-none-any.whl size=2976 sha256=44a8c4cd74ebd0d8165fab8bb1072ed43e4a81c70446abf77bbf86063508e215
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
Mounted at /content/drive
/content/drive/MyDrive/밑바닥부터 시작하는 딥러닝3


In [13]:
import numpy as np
import import_ipynb
import numpy as np
from Deep09 import as_array


### 13.2 Variable 클래스 수정

In [ ]:
# class Variable:
#   def __init__(self, data) :
#     self.data = data
#     self.grad = None
#     self.creator = None

#   def set_creator(self, func):
#     self.creator = func
  
#   def backward(self):
#     funcs = [self.creator]
#     while funcs:
#       f = funcs.pop()
#       x, y = f.input, f.output
#       x.grad = f.backward(y.grad)

#       if x.creator is not None:
#         funcs.append(x.creator)

      

In [39]:
class Variable:
  def __init__(self, data) :
    if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func):
    self.creator = func
  
  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)
    funcs = [self.creator]
    while funcs:
      f = funcs.pop()
      # output이 많으니까 차례차례 진행
      gys = [output.grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple) :
        # 튜플 변환
        gxs = (gxs,)
      for x, gx in zip(f.inputs, gxs):
        x.grad = gx
        if x.creator is not None:
          funcs.append(x.creator)

      

In [40]:
class Function : 
  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    # 별표를 붙여 언팩
    ys = self.forward(*xs)
    # 튜플이 아닌 경우 추가 지원
    if not isinstance(ys, tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys ]
    for output in outputs :
      output.set_creator(self)
    self.inputs = inputs
    self.outputs = outputs
    return outputs if len(outputs) > 1 else outputs[0]
  
  def forward(self,xs):
    raise NotImplementedError()

  def backward(self,gys):
    raise NotImplementedError()

In [41]:
class Add(Function):
  def forward(self,x0,x1):
    y = x0 + x1 
    return y
  def backward(self, gy):
    return gy,gy

In [42]:
def add(x0, x1):
  return Add()(x0,x1)

### 13.3 Square 클래스 구현

In [46]:
class Square(Function):
  def forward(self,x):
    y = x**2
    return y
    
  def backward(self, gy):
    x = self.inputs[0].data
    gx = 2*x*gy
    return gx

In [47]:
def square(x):
  return Square()(x)

In [48]:
if __name__ == '__main__':
  x = Variable(np.array(2.0))
  y = Variable(np.array(3.0))

  z = add(square(x), square(y))
  z.backward()
  print(z.data)
  print(x.grad)
  print(y.grad)

13.0
4.0
6.0
